In [ ]:
import os, sys

# clone rl-baselines3-zoo repo and checkout branch update/hf
# !pip install opencv-python # not headless version
rl_zoo3_path = "/github/rl-baselines3-zoo/"
model_folder = os.path.join(rl_zoo3_path, "rl-trained-agents/")
sys.path.insert(0, rl_zoo3_path)

import torch
# install torch with cuda support
device = torch.device('cuda')
num_threads = 16
torch.set_num_threads(num_threads)

sys.path.insert(0, "/github/rastermap/paper")
import qrdqn # has functions that wrap rl-baselines3-zoo and stable_baselines3
import fig5 

# path to directory with data etc
### *** CHANGE THIS TO WHEREEVER YOU ARE SAVING YOUR MODEL OUTPUTS ***
root = "/media/carsen/ssd2/rastermap_paper/"
# (in this folder we have a "simulations" folder)
os.makedirs(os.path.join(root, "simulations"), exist_ok=True)

env_ids = ["PongNoFrameskip-v4", "SpaceInvadersNoFrameskip-v4", 
           "EnduroNoFrameskip-v4", "SeaquestNoFrameskip-v4"]
n_seeds = 10


### run DQN

This code will run the DQN and save the activations for `n_seeds` worth of runs and save to the `simulations` folder.

In [ ]:
### IMPORTANT:
### need to run one game at a time then restart notebook because hooks stick around
# qrdqn.run_qrdqn(model_folder, root, env_id=env_ids[0], n_seeds=n_seeds, device=device)

### run rastermap and save

this function will run rastermap on the activations from each game and save to the `simulations` folder.

In [ ]:
### process qrdqn outputs
for env_id in env_ids:
    print(env_id)
    qrdqn.sort_spks(root, env_id)

### make figure

In [ ]:
# root path has folder "simulations" with saved results
# will save figures to "figures" folder
os.makedirs(os.path.join(root, "figures/"), exist_ok=True)
fig5.fig5(root)